In [1]:
!pip install kaggle

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jessicaisri","key":"6ffb8839e2f3cac673588f62174e100c"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp /content/kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d niyarrbarman/symptom2disease

  0% 0.00/43.6k [00:00<?, ?B/s]
100% 43.6k/43.6k [00:00<00:00, 59.6MB/s]


In [8]:
!unzip /content/symptom2disease.zip

Archive:  /content/symptom2disease.zip
  inflating: Symptom2Disease.csv     


# Import Libs

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read data

In [2]:
df = pd.read_csv('/content/Symptom2Disease.csv')

In [3]:
df.head(5)

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1200 non-null   int64 
 1   label       1200 non-null   object
 2   text        1200 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.2+ KB


In [5]:
df = df [['label', 'text']]

In [6]:
len(df['label'].unique())

24

# Prepare Data

In [7]:
import re

In [8]:
def clean_text(text):
  text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
  text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
  text = re.sub(r" +", ' ', text)
  return text

In [9]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

def change_synonym(sentence):
  tokens = nltk.word_tokenize(sentence)
  synonym = []
  for token in tokens:

        synonym_token = []
        for synset in wordnet.synsets(token, lang='eng'):
            for lemma in synset.lemmas(lang='eng'):
                synonym_token.append(lemma.name())

        if synonym_token:
            select_synonym = synonym_token[0]
            synonym.append(select_synonym)
        else:
            synonym.append(token)
  alter_text = ' '.join(synonym)
  return alter_text.replace('_', ' ')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
df2 = pd.DataFrame()
df2['clean_text'] = df['text'].apply(lambda x: change_synonym(clean_text(x)))
df2['label'] = df['label']

df['clean_text'] = df['text'].apply(clean_text)
df['clean_text'][0:5]

0    I have been experiencing a skin rash on my arm...
1    My skin has been peeling especially on my knee...
2    I have been experiencing joint pain in my fing...
3    There is a silver like dusting on my skin espe...
4    My nails have small dents or pits in them and ...
Name: clean_text, dtype: object

In [11]:
df = df[['clean_text', 'label']]

In [12]:
df = pd.concat([
    df,
    df2
])

In [13]:
df.tail()

,clean_text,label
1195,iodine 'm shaking and shaking all over . iodin...,diabetes
1196,particularly inch the crevice of my skin iodin...,diabetes
1197,iodine regularly experience these intense urge...,diabetes
1198,iodine rich person trouble breathing particula...,diabetes
1199,iodine constantly sneeze and rich person angst...,diabetes


In [92]:
change_synonym('I feel I have a skin condition. My face has rashes and little blisters around my nose. The wounds have turned red and painful, and a yellow liquid is oozing from them.')

'iodine feel iodine rich person angstrom skin condition . My face hour angle rash and little blister about my nose . The wound rich person turn red and painful , and angstrom yellow liquid be seepage from them .'

In [14]:
labels_encode, labels_categories = df.label.factorize()
labels_encode

array([ 0,  0,  0, ..., 23, 23, 23])

# Tokenize

In [15]:
!pip install bert-for-tf2

In [16]:
from tensorflow.keras import layers
import bert
import tensorflow_hub as hub

In [17]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1', trainable=False)

In [18]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [19]:
vocab_file

b'/tmp/tfhub_modules/88ac13afec2955fd14396e4582c251841b67429a/assets/vocab.txt'

In [20]:
len(tokenizer.vocab)

30522

In [21]:
def encode_sentence(sentence):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence))

In [22]:
encode_sentence('My cat loves fish')

[2026, 4937, 7459, 3869]

In [23]:
df

,clean_text,label
0,I have been experiencing a skin rash on my arm...,Psoriasis
1,My skin has been peeling especially on my knee...,Psoriasis
2,I have been experiencing joint pain in my fing...,Psoriasis
3,There is a silver like dusting on my skin espe...,Psoriasis
4,My nails have small dents or pits in them and ...,Psoriasis
...,...,...
1195,iodine 'm shaking and shaking all over . iodin...,diabetes
1196,particularly inch the crevice of my skin iodin...,diabetes
1197,iodine regularly experience these intense urge...,diabetes
1198,iodine rich person trouble breathing particula...,diabetes


In [24]:
data_clean = df.clean_text.values
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [25]:
data = [[sent, labels_encode[i], len(sent)] for i, sent in enumerate(data_inputs)]

In [26]:
import random
random.shuffle(data)
data.sort(key=lambda x: x[2])
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data if sent_lab[2] > 5]


In [27]:
import tensorflow as tf
all_data = tf.data.Dataset.from_generator(lambda: sorted_all, output_types = (tf.int32, tf.int32))

# Prepare training

In [28]:
BATCH_SIZE = 16
all_batched = all_data.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [29]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES

150

In [30]:
NB_BATCHES_TEST = NB_BATCHES // 10
NB_BATCHES_TEST

15

In [31]:
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [32]:
class DCNN(tf.keras.Model):

  def __init__(self,
               vocab_size,
               emb_dim=128,
               nb_filters = 50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.1,
               training=False,
               name="dcnn"):
    super(DCNN, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size, emb_dim)

    self.bigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 2,
                                padding='valid',
                                activation='relu')
    self.trigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 3,
                                padding='valid',
                                activation='relu')
    self.fourgram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 4,
                                padding='valid',
                                activation='relu')

    self.pool = layers.GlobalMaxPool1D()

    self.dense_1 = layers.Dense(units = FFN_units, activation='relu')
    self.dropout = layers.Dropout(rate=dropout_rate)

    # this data has 24 classes
    self.last_dense = layers.Dense(units=nb_classes, activation='softmax')

  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis = -1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output

In [59]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 36
NB_CLASSES = 24
DROPOUT_RATE = 0.2
NB_EPOCHS = 15

In [60]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters = NB_FILTERS,
            FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)

In [61]:

Dcnn.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['sparse_categorical_accuracy'])

In [50]:
checkpoint_path = '/content/'

In [62]:
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

In [63]:
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

In [64]:
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!')

In [65]:
class CallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    ckpt_manager.save()
    print("Checkpoint savet at {} ".format(checkpoint_path))

In [66]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   callbacks=[CallBack()])

Epoch 1/15
    135/Unknown - 18s 118ms/step - loss: 3.1522 - sparse_categorical_accuracy: 0.0546Checkpoint savet at /content/ 
135/135 [==============================] - 19s 120ms/step - loss: 3.1522 - sparse_categorical_accuracy: 0.0546
Epoch 2/15
135/135 [==============================] - 17s 126ms/step - loss: 3.0676 - sparse_categorical_accuracy: 0.1986
Epoch 3/15
135/135 [==============================] - 19s 137ms/step - loss: 2.9374 - sparse_categorical_accuracy: 0.3329
Epoch 4/15
135/135 [==============================] - 16s 118ms/step - loss: 2.7366 - sparse_categorical_accuracy: 0.4343
Epoch 5/15
135/135 [==============================] - 18s 134ms/step - loss: 2.4411 - sparse_categorical_accuracy: 0.4926
Epoch 6/15
135/135 [==============================] - 16s 119ms/step - loss: 2.0963 - sparse_categorical_accuracy: 0.5713
Epoch 7/15
135/135 [==============================] - 16s 121ms/step - loss: 1.7622 - sparse_categorical_accuracy: 0.6338
Epoch 8/15
135/135 [==========

In [68]:
import math
import numpy as np
def get_prediction(sentence):
    tokens = encode_sentence(clean_text(sentence))

    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False)

    results = output.numpy()
    results_array = np.array(results)

    indice_maior_valor = np.argmax(results_array)
    print(labels_categories[indice_maior_valor])

In [81]:
# @title Doctor UFO

text = "My tendency is to belch and burp frequently. I frequently get chest pain that spreads to my arm, jaw, and the back of my neck. There is pressure and stress in my chest." # @param {type:"string"}

get_prediction(text)

gastroesophageal reflux disease
